This notebook was written by Jorn Bruggeman for a workshop held 18-19 May 2021 as part of a contract from the [National Oceanography Centre](https://noc.ac.uk) to [Bolding & Bruggeman ApS](https://bolding-bruggeman.com/). It is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

In [ ]:
import datetime
import subprocess

import numpy
import scipy.stats
%matplotlib widget
from matplotlib import pyplot
from matplotlib.dates import num2date, date2num
import netCDF4
from ipywidgets import widgets

import sys
sys.path.append('../extern/fabm-mizer/python')
import mizer

# Function for converting from Equivalent Spherical Diameter (micrometer) to wet mass in g
def esd2mass(d): # d: equivalent spherical diameter in micrometer
    V = 4./3. * numpy.pi * (numpy.array(d) / 2e6)**3  # V: volume in m3
    return V * 1e6  # mass in g approximately equals volume in m3 multiplied by 1e6 (assumes density of 1000 kg/m3)

In [ ]:
# Run 1D simulation for NABE site (GOTM + FABM + MEDUSA)
# Forcing from https://igotm.bolding-bruggeman.com/
subprocess.check_call(['gotm'], cwd='../datasets/gotm-NABE', shell=True)

In [ ]:
# Read NetCDF
path = '../datasets/gotm-NABE/result.nc'
with netCDF4.Dataset(path) as nc:
    temp = nc['temp'][:, :, 0, 0]
    ZME = nc['pelagic_ZME'][:, :, 0, 0]
    ZMI = nc['pelagic_ZMI'][:, :, 0, 0]
    PHD = nc['pelagic_PHD'][:, :, 0, 0]
    PHN = nc['pelagic_PHN'][:, :, 0, 0]
    h = nc['h'][:, :, 0, 0]
    z = nc['z'][:, :, 0, 0]
    PT = ZME + ZMI + PHD + PHN
    numtime = nc['time'][:]
    numtime = numpy.broadcast_to(numtime[:, numpy.newaxis], z.shape)
    time = netCDF4.num2date(numtime, nc['time'].units, only_use_cftime_datetimes=False)

In [ ]:
# Plot total plankton concentration
fig, (ax1, ax2) = pyplot.subplots(figsize=(8, 7), nrows=2)
c = ax1.contourf(time, z, PT, numpy.linspace(0, 2.5, 51))
ax1.set_xlabel('time')
ax1.set_ylabel('depth (m)')
cb = fig.colorbar(c, ax=ax1)
cb.set_label('total plankton (mmol N/m³)')
ax1.set_ylim(-250, 0)
ax1.set_xlim(datetime.datetime(2018,1,1), datetime.datetime(2020,1,1))

# Plot inferred predator interaction depth
h_int = (PT*h).sum(axis=1)**2/(PT**2*h).sum(axis=1)

ax2.plot(time[:,0], h_int)
ax2.set_xlabel('time')
ax2.set_ylabel('interaction depth (m)')
ax2.set_xlim(datetime.datetime(2018,1,1), datetime.datetime(2020,1,1))
ax2.set_ylim(0, 250)
ax2.grid()

fig.savefig('h_int.png', dpi=300)

In [ ]:
# Calculate weights describing vertical predator distribution
weights = PT * h

# Average PFTs
PHN_ave = (weights * PHN).sum(axis=1) / weights.sum(axis=1) 
PHD_ave = (weights * PHD).sum(axis=1) / weights.sum(axis=1) 
ZMI_ave = (weights * ZMI).sum(axis=1) / weights.sum(axis=1) 
ZME_ave = (weights * ZME).sum(axis=1) / weights.sum(axis=1) 

# Plot
fig, (ax1, ax2, ax3, ax4) = pyplot.subplots(nrows=4, figsize=(8, 12))
ax1.plot(time[:,0], PHN[:, -1], label='surface')
ax1.plot(time[:,0], PHN_ave, label='predator average')
ax1.set_title('non-diatoms')
ax2.plot(time[:,0], PHD[:, -1], label='surface')
ax2.plot(time[:,0], PHD_ave, label='predator average')
ax2.set_title('diatoms')
ax3.plot(time[:,0], ZMI[:, -1], label='surface')
ax3.plot(time[:,0], ZMI_ave, label='predator average')
ax3.set_title('microzooplankton')
ax4.plot(time[:,0], ZME[:, -1], label='surface')
ax4.plot(time[:,0], ZME_ave, label='predator average')
ax4.set_title('mesozooplankton')
for ax in (ax1, ax2, ax3, ax4):
    ax.grid()
    ax.legend()
    ax.set_ylabel('concentration (mmol N/m³)')